## Import Postcode Data

In [ ]:
import pandas as pd 

postcodes = pd.read_csv("./postcodes.csv")

# show top 5 postcodes
print(postcodes.head())

# check postcode lengths
print(f"Postcode Lengths: {postcodes['Postcode'].apply(lambda s: len(s)).unique()}")


## Separate Outward, Inward and Area Codes

In [ ]:
import re

def split_postcode(s: str) -> tuple:
    """
    Splits postcode into outward and inward.
    """
    # replace double spaces in postcodes where area is only a single letter
    s = s.replace("  "," ")

    if " " not in s:
        outward = s[:4]
        inward = s[-3:]
    elif " " in s:
        outward, inward = s.split(" ")

    return outward, inward

postcodes[["Outward","Inward"]] = postcodes.apply(lambda x: split_postcode(x["Postcode"]), axis=1, result_type="expand")

# use regex to capture the first one or two letters in the outward code
postcodes["Area"] = postcodes["Outward"].apply(lambda s: re.match(r"([A-Z]{1,2})", s).groups()[0])

## Show Low Population Codes

In [ ]:
outward_grouped = (
    postcodes
    .groupby(['Outward'])
    .sum()
    .sort_values('Total')
    .reset_index()
)

area_grouped = (
    postcodes
    .groupby(['Area'])
    .sum()
    .sort_values('Total')
)

display(outward_grouped.query('Total < 20000'))
display(area_grouped.query('Total < 20000'))

## Mixed Outward-Area Approach

In [ ]:
# find outward codes with small population
bad_outward = outward_grouped.query("Total < 20000")['Outward']

# extract area code from bad_outward
bad_outward_area = [re.match(r"([A-Z]{1,2})", s).groups()[0] for s in bad_outward]

# create a dictionary from bad_outward and respective areas
outward_area_dict = dict(zip(bad_outward, bad_outward_area))

# outward codes with sufficient population map back to their original outward codes
for outward_code in postcodes['Outward'].unique():
    if outward_code not in outward_area_dict.keys():
        outward_area_dict[outward_code] = outward_code

# use dictionary to create new column called Outward_Area_Mix
postcodes['Outward_Area_Mix'] = postcodes['Outward'].map(outward_area_dict)

# show mixed approach where residents < 20000
(postcodes
    .groupby(["Outward_Area_Mix"])
    .sum()
    .sort_values('Outward_Area_Mix')
    .query('Total < 20000')
    .sort_values('Total')
)

## Outward Plot

In [ ]:
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

# import shape file
outward_shpfile = ("Distribution\Districts.shp")#
outward_geo = gpd.read_file(outward_shpfile)

# merge with outward_grouped
outward_merged = outward_geo.merge(outward_grouped, left_on="name", how="outer", right_on="Outward").fillna(0)

# create plots
fig, ax = plt.subplots(1,1,figsize=(25,25))
outward_merged[outward_merged["Total"] > 20000].plot(ax=ax, color="green")
outward_merged[outward_merged["Total"].between(5001,20000)].plot(ax=ax, color="orange")
outward_merged[outward_merged['Total'].between(1,5000)].plot(ax=ax, color="red")
outward_merged[outward_merged["Total"] == 0].plot(ax=ax, color="grey", alpha=0.3)

# show mirador for fun
mirador_coords = outward_merged[outward_merged['name'].str.contains('TD9')].geometry.centroid[2545].coords[0]
ax.annotate(text='Mirador Analytics', xy=mirador_coords,textcoords='offset pixels', xytext=(100,80), ha='center', color='black', arrowprops=dict(arrowstyle='-|>', facecolor='black'))

# add legend
red_patch = mpatches.Patch(color="red", label="Population <= 5k")
orange_patch = mpatches.Patch(color="orange", label="5k < Population <= 20k")
green_patch = mpatches.Patch(color="green", label="Population > 20k")
grey_patch = mpatches.Patch(color="grey", alpha=0.3, label="No Available Data")
plt.legend(handles=[red_patch, orange_patch, green_patch, grey_patch], bbox_to_anchor=(0.05, 0.98), loc='upper left')

# hide ticks
plt.xticks([]); plt.yticks([])

plt.show()

## Area Plot

In [ ]:
# import shape file
area_shpfile = ("Distribution\Areas.shp")
area_geo = gpd.read_file(area_shpfile)

# merge with area_grouped
area_merged = area_geo.merge(area_grouped, left_on="name", how="outer", right_on="Area").fillna(0)

# create plots
fig, ax = plt.subplots(1,1,figsize=(25,25))

area_merged[area_merged["Total"] > 20000].plot(ax=ax, color="green", edgecolor='white')
area_merged[area_merged["Total"].between(5001,20000)].plot(ax=ax, color="orange", edgecolor='white')
area_merged[area_merged['Total'].between(1,5000)].plot(ax=ax, color="red", edgecolor='white')
area_merged[area_merged["Total"] == 0].plot(ax=ax, color="grey", alpha=0.3, edgecolor='white')

# annotate areas
area_merged.apply(lambda x: ax.annotate(text=x['name'], xy=x.geometry.centroid.coords[0], ha='center', color="White"),axis=1)

# show mirador for fun
mirador_coords = outward_merged[outward_merged['name'].str.contains('TD9')].geometry.centroid[2545].coords[0]
ax.annotate(text='Mirador Analytics', xy=mirador_coords,textcoords='offset pixels', xytext=(100,80), ha='center', color='black', arrowprops=dict(arrowstyle='-|>', facecolor='black'))

# add legend
red_patch = mpatches.Patch(color="red", label="Population <= 5k")
orange_patch = mpatches.Patch(color="orange", label="5k < Population <= 20k")
green_patch = mpatches.Patch(color="green", label="Population > 20k")
grey_patch = mpatches.Patch(color="grey", alpha=0.3,  label="No Available Data")
plt.legend(handles=[red_patch, orange_patch, green_patch, grey_patch], bbox_to_anchor=(0.05, 0.98), loc='upper left')

# hide ticks
plt.xticks([]); plt.yticks([])

plt.show()